# Quadratic Unconstrained Binary Optimization (QUBO)
## Introduction
The Quadratic Unconstrained Binary Optimization (QUBO) is a foundational problem type used in many fields to formulate discrete combinatorial optimization problems. Generically, a Qubo is defined by linear and quadratic terms, but since $x^2 = x$ when $x \in \{0,1\}$, we can simplify the optimization expression as such $f(x) = \sum_{i} \sum_{j} J_{ij} x_i x_j$, where $f: \mathbb{B}^n \rightarrow \mathbb{R}$. Note that the coefficients naturally encodes as a square symmetric matrix, so that $f(x) = x^t Q x$, where $Q$ has entries $Q_{ij}$. The goal of the optimization problem is to find the binary vector, $x^{*}$, that minimizes $f(x)$, 
$x^{*} = \min_{x} x^t Q x$. 


In [1]:
import numpy as np
from qci_client import QciClient
client = QciClient()

## Uploading a QUBO job
### Data format
To upload a square symmetric matrix or Qubo, we encode it in a sparse matrix format as shown below. We use Numpy array notation. 


In [2]:
Q = np.array([[0, -1.5, 0.5], [-1.5, 0, 0], [0.5, 0, 0]]) 

In [3]:
qubo_data = {
  "data": Q,
  "file_name": "smallest_objective.json", # can be any short string
  "num_variables": 3, # number of rows
  "file_type": "qubo" # defines the data type, 'qubo' in this case
}



### Uploading
To upload the matrix encoded above in `qubo_data`, we use the the `qci_client` imported previously. The following line 


In [ ]:
response_json = qci.upload_file(qubo_data)

The response contains a file_id for the uploaded file. This id is provided when a job is run, along a few other parameters (see #Running). Note: the same `file_id` can be used multiple times to run a problem repeatedly. This enables an "upload once, run many times" scheme, which is especially useful for job types in which parameter searches may be involved.
Triggering a job requires two items: first a job body that contains essential and optional metadata for the job, and second, the type of job a user wants to run. 

## Running a QUBO job
### Job body
This section defines the job body for a Qubo job.  



In [ ]:

job_body = {
"job_name": "test-job", # required, can be any string
"job_tags": ["foo", "bar"], # optional, useful for tracking different jobs 
"params": params, # dictionary containing job parameters
"qubo_file_id": response_json["file_id"] # string id returned from file upload
}

Additional parameters that can be included in the job using the params dictionary. Defaults are listed first, optional parameters in the complete list below. First, here is a params example:

In [ ]:
params = {
"sampler_type": "csample", 
"n_samples": 100
}


### Parameters list 
sampler_type: "csample", "eqc1", "eqc2"
n_samples: defaults to 100 for csample jobs; defaults to 1 for eqc1 and eqc2 jobs. Note that each EQC sample is a job run in serial so multiple samples may incur a large overhead for large jobs. 

### Submitting a job
To run a job, the API needs the job_body with the file_id from the upload step and the job 
result = qci.process_job(job_body=job_json, job_type="sample-qubo")
The process_job method is not asynchronous and includes a polling step within it. If the user wants more control the polling stage within process_job will serve as a good example of the few steps needed to create a sequence of asynchronous polling steps.
The above call will return a result dictionary. To print the results:


In [ ]:

results = result["results"]
print(results)


Below we show how to query the result object if an error occurs:
    



In [ ]:
if result['job_info']['details']['status'] == "ERROR":
    print(result['job_info']['details']['status'])
    print(result['job_info']['results']['error'])
